Читаем данные таблицы

In [ ]:
import pandas as pd 

data = pd.read_csv("Datafiniti_Fast_Food_Restaurants.csv")

Посмотрим первые записи

In [ ]:
data.head()

Перечислим первые три значения колонки `dateAdded` через запятую и пробел в том порядке, в котором они были выведены

In [ ]:
new_data = data['dateAdded'].head(3)
list(new_data)

Посмотрим, какой тип имеют колонки `city` и `latitude`

In [ ]:
print(f"city - {data['city'].dtype}")
print(f"latitude - {data['latitude'].dtype}")

Посмотрим описание данных с помощью `describe`. Вывелись `latitude` и `longitude`, потому что это единственные численные колонки

In [ ]:
data.describe()

Проверим, есть ли пропуски в данных 

In [ ]:
data.isna().any().any()

При наличи пропусков можно воспользоваться методом `dropna`. Метод `dropna` в **Pandas** используется для удаления строк или столбцов, содержащих пропущенные значения

Отберем заведения, которые находятся в городе `Detroit`.  Укажем индексы, под которыми полученные заведения расположены в датафрейме, через запятую и пробел в порядке возрастания. 

In [ ]:
print(list(data[data['city'] == 'Detroit'].index))

Отберем заведения `Taco Bell` в городе `Detroit`.
Укажем индексы этих заведений через запятую и пробел в порядке возрастания.

In [ ]:
print(list(data[(data['name'] == 'Taco Bell') & (data['city'] == 'Detroit')].index))

Найдем заведения `Taco Bell` или заведения, которые находятся в городе **Нью-Йорк**. При этом обязательно, чтобы в названии меню не было `Volcano Taco` и `Fresco Soft Taco` (именно таких значений колонки). Данные сохраним в датафрейм `result`.

In [ ]:
result = data[((data['name'] == 'Taco Bell') | (data['city'] == 'New York'))
               & (~data['name'].isin(['Volcano Taco', 'Fresco Soft Taco']))]
result

Попробуемсделать `data['categories']`, `data[['categories']]`, посмотрим на тип и на значения их элементов.

In [ ]:
print(data['categories'])
print(type(data['categories']))

In [ ]:
print(data[['categories']])
print(type(data[['categories']]))

- В коде вида `датафрейм[одна_строка]` pandas видит запрос на получение **одной** колонки и возвращает `Series`.
- В коде вида `датафрейм[список_из_строк]`, где в списке лежит только один элемен, pandas видит запрос на получение **нескольких** колонок. `Series` по своей сущности может содержать только одну колонку, поэтому pandas в качестве результата выдаст `DataFrame`, куда войдут только те строки, которые вы заказали.

Посчитаем, сколько ресторанов *Taco Bell* находится в каждом городе. Отберем 5 городов, в которых ресторан встречается чаще всего. Количество ресторанов для этих городов с их названиями сохраним в виде объекта `pd.Series` в переменную `result`.

In [ ]:
result = data[data['name'] == 'Taco Bell'].groupby('city').agg({'name': 'count'}) # считаем количество ресторанов в каждом городе
result = result.sort_values(by='name', ascending=False)[:5] # берем 5 городов, где ресторан встречается чаще всего
result.squeeze() 
# Метод squeeze() в Pandas используется для удаления ненужных измерений из DataFrame или Series. Он особенно полезен, 
# когда вы работаете с объектами, у которых есть только одно измерение, но формально они всё ещё остаются DataFrame или Series.

Найдем рестораны, открытые в октябре.
Выведем индексы первых пяти ресторанов через запятую и пробел в порядке возрастания.

In [ ]:
data_with_datetime = data.copy()
data_with_datetime['dateAdded'] = pd.to_datetime(data_with_datetime['dateAdded']) # создадим копию и в ней превратим колонку в pd.datetime
result = data_with_datetime[data_with_datetime['dateAdded'].dt.month == 10]
print(list(result.index[:5]))

Сделаем группировку по месяцу открытия ресторана (переменная `dateAdded`). Сколько ресторанов было открыто в каждом месяце? Рассчитаем по количеству уникальных `id` ресторанов в каждом месяце. Значения `dateAdded` должны при этом быть индексами. Результирующий датафрейм сохраним в `result`

In [ ]:
result = data_with_datetime.groupby(data_with_datetime['dateAdded'].dt.month).agg({'id': 'nunique'})
result.index = pd.to_datetime(result.index, format='%m').strftime('%B')
result
# format='%m' указывает, что месяцы в result.index представлены в формате "%m", 
# то есть как двузначные числа (например, "01" для января, "02" для февраля и т.д.).

# .strftime(...): Это метод, доступный для объектов datetime в Python. 
# Он позволяет форматировать дату в строку с помощью специальных кодов формата. 
# '%B' - код формата указывает на полное название месяца (например, "January", "February" и т.д.). 

Добавим столбец `update_delta` с количеством целых дней, полученных от разницы между `dateUpdated` и `dateAdded`.
Потом для каждого города найдем:
- среднее по переменной `update_delta`
- широту самой северной закусочной (чем севернее расположена закусочная, тем больше будет значение широты)
Названия городов не должны быть индексами. Сохраним результирующий датафрейм в переменную `result`.  Также сохраним в переменную `det_mean` среднее значение `update_delta` по городу *Detroit*.


In [ ]:
data_with_update_delta = data_with_datetime.copy()
data_with_update_delta['dateUpdated'] = pd.to_datetime(data_with_update_delta['dateUpdated'])
data_with_update_delta['update_delta'] = (data_with_update_delta['dateUpdated'] - data_with_update_delta['dateAdded']).dt.days

mean_update_delta = data_with_update_delta.groupby('city').agg({'update_delta': 'mean'})['update_delta']
max_latitude = data_with_update_delta.groupby('city').agg({'latitude': 'max'}).squeeze()

result = pd.DataFrame({'mean_update_delta': mean_update_delta, 'max_latitude': max_latitude})
det_mean = mean_update_delta['Detroit']
print(det_mean)
result

Отберем рестораны, у которых в колонке `categories` упоминается *Pizza*
Укажем индекс 3-го элемента, полученного после фильтрации


In [ ]:
data[data['categories'].str.contains('Pizza')].index[2]

Добавим столбец `update_delta`, куда будет записана разница в днях между `dateUpdated` и `dateAdded`.  Посчитаем среднее и медиану получившейся колонки.
Перечислим значения через пробел (сначала среднее, затем медиану), округлив до 2 знаков. 


In [ ]:
mean = round(data_with_update_delta['update_delta'].mean(), 2)
median = round(data_with_update_delta['update_delta'].median(), 2)
print(mean, median)

Отоберем записи, у которых более 2 категорий, далее сгруппируем их по провинциям и подсчитаем минимальную `longitude` в каждой. Округлим значения `longitude` до 3-х знаков после запятой.
Далее сохраним результирующий датафрейм в файл в формате **csv** с сепаратором `;`. Сам датафрейм должен содержать две колонки: `province` и `longitude`


In [ ]:
many_categories = data[data['categories'].str.split(',').apply(len) > 2]
result = many_categories.groupby('province').agg({'longitude': 'min'})
result['longitude'].apply(lambda x: round(x, 3))
result.to_csv('output/result.csv', sep=';')
result